## Path Metadata Validation Example

This notebook demonstrates how to check that the metadata associated with an experiment will be interpreted correctly based on an experiment configuration file.

In [3]:
%run -m celldom.nb.logging
import os
import os.path as osp
import celldom
from celldom.config import experiment_config

Set the experiment configuration to be loaded:

In [4]:
celldom.get_repo_dir()

'/lab/repos/celldom'

In [6]:
exp_config_path = osp.join(celldom.get_repo_dir(), 'config', 'experiment', 'experiment_example_G3.yaml')
exp_config_path

'/lab/repos/celldom/config/experiment/experiment_example_G3.yaml'

In [25]:
!cat $exp_config_path

name: G3 Test Experiment
chip: {name: chip_G3}
cytometer: {name: cytometer_01}
acquisition:
  magnification: 16
  reflection: false
metadata:
  # Example file path: */_2018.06.14 EXP SUM Control 0.1uM with 5mL gravity/2018.06.14 White 3 Control/2018.06.14 White 3 Control 0 hr/BFF_16X_St_000_Apt_000_201806150024.tif
  fields:
    datetime:
      regex: '[0-9]{12}'
      format: '%Y%m%d%H%M'
    pump: 'Elveflow|gravity'
    color: 'Orange|Red|White|Pink|Green|Blue|Yellow'
    condition: '1|3'
    hours: '[0-9]{1,3}'
    cohort: 'Control|[0-9\.]{3,}uM'
    street: '[0-9]{3}'
    apartment: '[0-9]{3}'
  path_format: '.*/.*{pump}/.*/.* {color} {condition} {cohort} {hours} hr/BFF_16X_St_{street}_Apt_{apartment}_{datetime}.tif'

#### Load Configuration

In [26]:
exp_config = experiment_config.ExperimentConfig(celldom.read_config(exp_config_path))

In [46]:
exp_config.conf

{'acquisition': {'magnification': 16, 'reflection': False},
 'chip': {'name': 'chip_G3'},
 'cytometer': {'name': 'cytometer_01'},
 'metadata': {'fields': {'apartment': '[0-9]{3}',
   'cohort': 'Control|[0-9\\.]{3,}uM',
   'color': 'Orange|Red|White|Pink|Green|Blue|Yellow',
   'condition': '1|3',
   'datetime': {'format': '%Y%m%d%H%M', 'regex': '[0-9]{12}'},
   'hours': '[0-9]{1,3}',
   'pump': 'Elveflow|gravity',
   'street': '[0-9]{3}'},
  'path_format': '.*/.*{pump}/.*/.* {color} {condition} {cohort} {hours} hr/BFF_16X_St_{street}_Apt_{apartment}_{datetime}.tif'},
 'name': 'G3 Test Experiment'}

#### Test Path Parsing

In [47]:
# Create a path to test parsing of metadata properties on
test_path = \
    'JeffsData/_2018.06.14 EXP SUM Control 0.1uM with 5mL gravity/2018.06.14 White 3 Control/' \
    '2018.06.14 Pink 3 1.0uM 0 hr/BFF_16X_St_001_Apt_016_201806150024.tif'
test_path

'JeffsData/_2018.06.14 EXP SUM Control 0.1uM with 5mL gravity/2018.06.14 White 3 Control/2018.06.14 Pink 3 1.0uM 0 hr/BFF_16X_St_001_Apt_016_201806150024.tif'

In [48]:
# Test that the path can be parsed successfully
exp_config.parse_path(test_path)

{'apartment': '016',
 'cohort': '1.0uM',
 'color': 'Pink',
 'condition': '3',
 'datetime': Timestamp('2018-06-15 00:24:00'),
 'hours': '0',
 'pump': 'gravity',
 'street': '001'}